In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import urllib.request
import numpy as np 
from os import listdir
import matplotlib.pyplot as plt
import cv2 as cv

In [118]:
col_names = ['group', 'subgroup', 'subgroup url', 'case ID', 'case description', 'img url', 'img name', 'img number', 'sex', 'age', 'body part']

data = pd.DataFrame(columns=col_names)

In [61]:
# read in file wth the images that are ultrasound images only
df_labels = pd.read_csv('HMSS_labels_rem.csv')

In [102]:
# make lists of all sections from the ultrasound only file
groups = df_labels['group'].unique().tolist()

# make list of all subgroups
subgroups = df_labels.groupby('group')['subgroup'].unique().apply(list)

# make list of all images per subgroup
imgs = df_labels['img name'].tolist()

In [ ]:
# handle request and return as parsed HTML
def request_data(url):
    """
    Request data from HMSS website and parse with beautifulsoup.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

# download images
def download_imgs(urllink, tgt_path, filename):
    # full path: tgt_path + filename
    full_tgt_path = os.path.join(tgt_path, filename)
    # download image to the folder path
    urllib.request.urlretrieve(urllink, full_tgt_path)
    print('{} saved to {}.'.format(filename, full_tgt_path))

In [116]:
"""
Scraping the data from HMSS website for section 4 (4.1-4.5) 
"""

base_url = 'https://www.ultrasoundcases.info/'
INIT_URL = "https://www.ultrasoundcases.info/cases/head-and-neck/"
init_soup = request_data(INIT_URL)
first_level = init_soup.find_all("div", {"class": "candidate-filter"})
tgt_path = 'MedAI-MAE/data/imgs'
# make sure file directory exists
if not os.path.exists(tgt_path):
    os.makedirs(tgt_path)

img_num = 0 # number images starting from 0
# go through all first level categories (first level = body system (EX. 4 Head and Neck))
for item_l1 in first_level:
    # get the title from h4>a
    item_l1_title = item_l1.find("h4").find("a").text
    # only fetch section 4 items
    if (item_l1_title != 'Head and Neck'):
        continue
    
    print('Fetching', item_l1_title)
    
    # get the second level categories
    second_level = item_l1.find("ul").find_all("li")
    
    # iterate through each item in the second level (second level = group of body system (EX. 4.1 Thyroid Gland))
    for item_l2 in second_level:
        item_l2_title = item_l2.find("a").text
        item_l2_url_title = item_l2.find("a")['href']
        item_l2_id = item_l2.find("a")["data-id"] 

        # check if the group title is in the ultrasound only file
        if item_l2_title not in groups:
            continue # go to the next group

        # get the subgroups for this group
        l2_subgroups = subgroups[item_l2_title]

        print("Fetching", item_l2_title, item_l2_id)

        third_level = requests.get(
            "https://www.ultrasoundcases.info/api/cases/list/" + str(item_l2_id)
        ).json()
        
        # iterate through each item in the third level (third level = subgroup of group (EX. 4.1.1 Thyroid Congenital Abnormalities))
        for item_l3 in third_level: 
            item_l3_title = item_l3["title"]
            item_l3_urltitle = item_l3["urltitle"]
            item_l3_id = item_l3["id"]

            # check if subgroup is in the ultrasound only file
            if item_l3_title not in l2_subgroups:
                continue # go to the next subgroup

            print("Fetching", item_l3_title, item_l3_id)

            fourth_level = requests.post(
                "https://www.ultrasoundcases.info/api/cases/list/" +  str(item_l3_id) + '/',
                data={"type": "subsubcat"}
            ).json()

            # iterate through each item in the fourth level (fourth level = specific cases of subgroup (EX. Hypoplasia of the left thyroid lobe))
            for item_l4 in fourth_level:
                item_l4_id = item_l4['id']
                item_l4_title = item_l4['title']
                item_l4_subtitle = item_l4['subtitle']
                item_l4_thumb = item_l4['thumb']
                item_l4_urltitle = item_l4['urltitle']

                print("Fetching", item_l4_title, item_l4_id, item_l4_urltitle)

                item_l5_soup = request_data(
                    "https://www.ultrasoundcases.info/{}/".format(item_l4_urltitle)
                )

                # img information
                fifth_level = item_l5_soup.find("div", {"class": "portfolio"})
                fifth_level = fifth_level.find_all("img")

                # patient details
                l5_details = item_l5_soup.find("div", {"class": "information"})
                l5_h4 = l5_details.find("h4")
                
                # check if details about case exist
                if(l5_h4.text == 'Details'):
                    l5_patient_info = l5_details.find_all("li")
                    for i, li in enumerate(l5_patient_info):
                        details = ''.join(li.find_all(string=True, recursive=False)).strip()
                        if i == 0:
                            l5_sex = details
                        if i == 1:
                            l5_age = details
                        if i == 2:
                            l5_body_part = details
                else:
                    l5_sex = np.nan
                    l5_age = np.nan
                    l5_body_part = np.nan
                    

                # iterate through each item in the case level (fifth level = image details for the specific case)
                for item_l5 in fifth_level:  # case level

                    item_l5_img_url_src = item_l5["src"]
                    item_l5_img_url = "https://www.ultrasoundcases.info" + item_l5_img_url_src

                    # get image name
                    img_name = item_l5_img_url.split('/')
                    item_l5_img_name = img_name[-1]

                    # check if image is in the ultrasound images only file
                    if item_l5_img_name not in imgs:
                        continue # go to the next image

                    img_soup = request_data(
                        item_l5_img_url
                    )

                    print("Fetching", item_l5_img_url)

                    # prep data for data frame
                    item_l3_url = base_url + item_l2_url_title + item_l3_urltitle

                    # img number
                    img_num_name = f"{img_num:04d}"

                    # create row for dataframe
                    data.loc[len(data.index)] = [item_l2_title, item_l3_title, item_l3_url, item_l4_id, item_l4_subtitle, item_l5_img_url, item_l5_img_name, img_num_name, l5_sex, l5_age, l5_body_part]

                    # download images here
                    #download_imgs(item_l5_img_url, tgt_path, item_l5_img_name)

                    # add one to the image number
                    img_num += 1


Fetching Head and Neck
Fetching 4.1 Thyroid gland 94
Fetching 4.1.2 Thyroid benign lesions 276
Fetching Thyroid benign lesions 3428 thyroid-benign-lesions-7944
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71396-Afbeelding1.jpg
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71397-Afbeelding2.jpg
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71398-Afbeelding3.jpg
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71399-Afbeelding4.jpg
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71400-Afbeelding5.jpg
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71401-Afbeelding6.jpg
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71402-Afbeelding7.jpg
Fetching Thyroid benign lesions 3417 thyroid-benign-lesions-6881
Fetching https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/54946-Afbeelding1.jpg
Fetchin

KeyboardInterrupt: 

In [123]:
df_labels.head(1)

,group,subgroup,subgroup url,case description,img url,img name,sex,age,body part,label,dataset
0,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71396-Afbeelding1.jpg,Female,43,Thyroid,benign,HMSS


In [126]:
tgt_group = '4.1 Thyroid gland'
tgt_subgroup = '4.1.2 Thyroid benign lesions'
tgt_subgroupurl = 'https://www.ultrasoundcases.info//cases/head-and-neck/thyroid-gland/thyroid-benign-lesions'
tgt_casedescription = 'Thyroid lesion that biopsy proved to be a follicular proliferation'
tgt_imgurl = 'https://www.ultrasoundcases.info/clients/ultrasoundcases/uploads/71396-Afbeelding1.jpg'
tgt_imgname = '71396-Afbeelding1.jpg'

result_row = df_labels[(df_labels['group'] == tgt_group) & (df_labels['subgroup'] == tgt_subgroup) & (df_labels['subgroup url'] == tgt_subgroupurl) & (df_labels['case description'] == tgt_casedescription) & (df_labels['img url'] == tgt_imgurl) & (df_labels['img name'] == tgt_imgname)]

label = result_row['label']
print(label)

0    benign
Name: label, dtype: object


In [115]:
data

,group,subgroup,subgroup url,case ID,case description,img url,img name,img number,sex,age,body part
0,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71396-Afbeelding1.jpg,0000,Female,43,Thyroid
1,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71397-Afbeelding2.jpg,0001,Female,43,Thyroid
2,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71398-Afbeelding3.jpg,0002,Female,43,Thyroid
3,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71399-Afbeelding4.jpg,0003,Female,43,Thyroid
4,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71400-Afbeelding5.jpg,0004,Female,43,Thyroid
5,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71401-Afbeelding6.jpg,0005,Female,43,Thyroid
6,4.1 Thyroid gland,4.1.2 Thyroid benign lesions,https://www.ultrasoundcases.info//cases/head-a...,3428,Thyroid lesion that biopsy proved to be a foll...,https://www.ultrasoundcases.info/clients/ultra...,71402-Afbeelding7.jpg,0006,Female,43,Thyroid


In [9]:
# write dataframe to csv file
#data.to_csv('section4.csv', index=False)